In [1]:
import os
import urllib.request
import logging

In [2]:
station_code_dict = {
    "PANC": "USW00026451", # Anchorage 
    "KBOI": "USW00024131", # Boise  
    "KORD": "USW00094846", # Chicago
    "KDEN": "USW00003017", # Denver 
    "KDTW": "USW00094847", # Detroit
    "PHNL": "USW00022521", # Honolulu 
    "KIAH": "USW00012960", # Houston
    "KMIA": "USW00012839", # Miami 
    "KMSP": "USW00014922", # Minneapolis 
    "KOKC": "USW00013967", # Oklahoma City 
    "KBNA": "USW00013897", # Nashville 
    "KJFK": "USW00094789", # New York 
    "KPHX": "USW00023183", # Phoenix 
    "KPWM": "USW00014764", # Portland ME
    "KPDX": "USW00024229", # Portland OR 
    "KSLC": "USW00024127", # Salt Lake City
    "KSAN": "USW00023188", # San Diego 
    "KSFO": "USW00023234", # San Francisco 
    "KSEA": "USW00024233", # Seattle 
    "KDCA": "USW00013743", # Washington DC
}

In [3]:
data_path_url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/all/"

# Directory to save downloaded files
raw_noaa_cache = "data/noaa"

# Ensure the directory exists
os.makedirs(raw_noaa_cache, exist_ok=True)

# URL to download data from
data_path_url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/all/"

# Setup logging
logging.basicConfig(level=logging.INFO)

# Loop through the station codes and download the corresponding files
for station_code, file_name in station_code_dict.items():
    url = f"{data_path_url}{file_name}.dly"
    try:
        # Download the file and save it
        urllib.request.urlretrieve(url, os.path.join(raw_noaa_cache, f"{station_code}.dly"))
        logging.info(f"Successfully scraped data for: {station_code}")
    except Exception as e:
        logging.error(f"Failed to download data for {station_code}: {e}")

INFO:root:Successfully scraped data for: PANC
INFO:root:Successfully scraped data for: KBOI
INFO:root:Successfully scraped data for: KORD
INFO:root:Successfully scraped data for: KDEN
INFO:root:Successfully scraped data for: KDTW
INFO:root:Successfully scraped data for: PHNL
INFO:root:Successfully scraped data for: KIAH
INFO:root:Successfully scraped data for: KMIA
INFO:root:Successfully scraped data for: KMSP
INFO:root:Successfully scraped data for: KOKC
INFO:root:Successfully scraped data for: KBNA
INFO:root:Successfully scraped data for: KJFK
INFO:root:Successfully scraped data for: KPHX
INFO:root:Successfully scraped data for: KPWM
INFO:root:Successfully scraped data for: KPDX
INFO:root:Successfully scraped data for: KSLC
INFO:root:Successfully scraped data for: KSAN
INFO:root:Successfully scraped data for: KSFO
INFO:root:Successfully scraped data for: KSEA
INFO:root:Successfully scraped data for: KDCA


In [1]:
import pandas as pd

# Define the column names and their respective column positions
columns = ['ID', 'YEAR', 'MONTH', 'ELEMENT'] + [f'VALUE{i}' for i in range(1, 32)] + [f'MFLAG{i}' for i in range(1, 32)] + [f'QFLAG{i}' for i in range(1, 32)] + [f'SFLAG{i}' for i in range(1, 32)]

# Define the fixed column positions for each variable
column_positions = [
    (0, 11),  # ID (1-11)
    (11, 15),  # YEAR (12-15)
    (15, 17),  # MONTH (16-17)
    (17, 21)  # ELEMENT (18-21)
] + [(i*5+21, i*5+26) for i in range(31)] * 3  # VALUE1 to VALUE31, MFLAG1 to MFLAG31, QFLAG1 to QFLAG31, SFLAG1 to SFLAG31

# Function to parse a single line of the file
def parse_line(line):
    data = {}
    
    # Extract the values for each column based on their positions
    data['ID'] = line[0:11].strip()
    data['YEAR'] = int(line[11:15].strip())
    data['MONTH'] = int(line[15:17].strip())
    data['ELEMENT'] = line[17:21].strip()

    # Extract VALUE, MFLAG, QFLAG, SFLAG columns
    for i in range(31):
        value_str = line[21 + i*5:26 + i*5].strip()
        
        # Try to convert the value to an integer, but handle non-numeric values
        try:
            data[f'VALUE{i+1}'] = int(value_str) if value_str else None  # Assign None if the value is empty
        except ValueError:
            data[f'VALUE{i+1}'] = None  # If there's a conversion error, assign None
        
        # Extract MFLAG, QFLAG, SFLAG (which should be single characters)
        data[f'MFLAG{i+1}'] = line[26 + i*5:27 + i*5].strip()
        data[f'QFLAG{i+1}'] = line[27 + i*5:28 + i*5].strip()
        data[f'SFLAG{i+1}'] = line[28 + i*5:29 + i*5].strip()
    
    return data

# Read the file and parse all lines into a list of dictionaries
file_path = 'data/noaa/KBNA.dly'  # Replace with your .dly file path
records = []
with open(file_path, 'r') as f:
    for line in f:
        record = parse_line(line)
        records.append(record)

# Convert the list of records to a DataFrame
df = pd.DataFrame(records)

# Show the first few rows of the DataFrame
print(df.head())



            ID  YEAR  MONTH ELEMENT  VALUE1 MFLAG1 QFLAG1 SFLAG1  VALUE2  \
0  USW00013897  1948      1    TMAX     183                    0     0.0   
1  USW00013897  1948      1    TMIN     -11                    0     0.0   
2  USW00013897  1948      1    PRCP     678                    0     0.0   
3  USW00013897  1948      1    SNOW       0      T             X     NaN   
4  USW00013897  1948      1    SNWD       0                    X     NaN   

  MFLAG2  ... QFLAG29 SFLAG29  VALUE30 MFLAG30 QFLAG30 SFLAG30  VALUE31  \
0         ...               2     22.0               0              0.0   
1      -  ...       1       7   -178.0               0              NaN   
2         ...                      0.0               0              0.0   
3         ...                      0.0               X              NaN   
4         ...       1       0    102.0               X              NaN   

  MFLAG31 QFLAG31 SFLAG31  
0       7       8          
1       5       6          
2       

In [2]:
df

,ID,YEAR,MONTH,ELEMENT,VALUE1,MFLAG1,QFLAG1,SFLAG1,VALUE2,MFLAG2,...,QFLAG29,SFLAG29,VALUE30,MFLAG30,QFLAG30,SFLAG30,VALUE31,MFLAG31,QFLAG31,SFLAG31
0,USW00013897,1948,1,TMAX,183,,,0,0.0,,...,,2,22.0,,0,,0.0,7,8,
1,USW00013897,1948,1,TMIN,-11,,,0,0.0,-,...,1,7,-178.0,,0,,NaN,5,6,
2,USW00013897,1948,1,PRCP,678,,,0,0.0,,...,,,0.0,,0,,0.0,,0,T
3,USW00013897,1948,1,SNOW,0,T,,X,NaN,,...,,,0.0,,X,,NaN,,0,T
4,USW00013897,1948,1,SNWD,0,,,X,NaN,,...,1,0,102.0,,X,,NaN,5,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17420,USW00013897,2024,11,WSF2,98,,,W,NaN,,...,9,9,9999.0,,,-,-99.0,9,9,
17421,USW00013897,2024,11,WSF5,125,,,W,NaN,,...,9,9,9999.0,,,-,-99.0,9,9,
17422,USW00013897,2024,11,WT01,1,,,W,NaN,9,...,9,9,9999.0,,,-,-99.0,9,9,
17423,USW00013897,2024,11,WT03,-9999,,,,-9.0,9,...,9,9,9999.0,,,-,-99.0,9,9,


In [27]:
df.columns

Index(['ID', 'YEAR', 'MONTH', 'ELEMENT', 'VALUE1', 'MFLAG1', 'QFLAG1',
       'SFLAG1', 'VALUE2', 'MFLAG2',
       ...
       'QFLAG29', 'SFLAG29', 'VALUE30', 'MFLAG30', 'QFLAG30', 'SFLAG30',
       'VALUE31', 'MFLAG31', 'QFLAG31', 'SFLAG31'],
      dtype='object', length=128)

### Day-of-Year Averages:
Calculate the average temperature for each day of the year across all years for each city. This provides a seasonal temperature baseline for each day (or month) that can help in pre

In [ ]:
def process_noaa(station):
    try:
        # Construct the path for the raw NOAA data file
        noaa_path = os.path.join(utils.raw_noaa_cache, f"{station}.dly")
        
        # Read the data from the file
        noaa_data = read_noaa_data_file(noaa_path)
        
        # Create the directory for processed data if it doesn't exist
        os.makedirs(utils.processed_noaa_cache, exist_ok=True)
        
        # Construct the output path for the processed data
        noaa_out_path = os.path.join(utils.processed_noaa_cache, f"{station}.csv")
        
        # Save the processed data to a CSV file
        noaa_data.to_csv(noaa_out_path)
        
        # Log the successful processing
        logging.info(f"Processed data for station: {station}")
    
    except Exception as e:
        # Log any errors encountered
        logging.error(f"Failed to process data for {station}: {e}")

# Iterate over the list of stations in utils.stations
for station in utils.stations_list:
    process_noaa(station)
